Week 3 (26 April, 2021)
Practical Course: Learning For Self-Driving Cars and Intelligent Systems
Winter Semester 2020-2021
Technical University of Munich
https://vision.in.tum.de/teaching/ss2021/intellisys_ss2021
Example from the Slides 

In [7]:
import torch
import numpy as np

In [8]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "graph.png")

References:
 

1.   https://pytorch-geometric.readthedocs.io/en/latest/notes/create_gnn.html
2.   https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-7d2250723780 


         
 

In [5]:
A = np.array([
    [0, 1, 1, 0],
    [1, 0, 0, 1], 
    [0, 0, 0, 1],
    [1, 0, 0, 0]],
    dtype=float
)
X = np.array([
            [i+1, (i+1)**2]
            for i in range(A.shape[0])
        ], dtype=float)
print(X)
print(A)
print(A @ X)

[[ 1.  1.]
 [ 2.  4.]
 [ 3.  9.]
 [ 4. 16.]]
[[0. 1. 1. 0.]
 [1. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]
[[ 5. 13.]
 [ 5. 17.]
 [ 4. 16.]
 [ 1.  1.]]


In [6]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GraphConvLayer(MessagePassing):
    def __init__(self):
        super(GraphConvLayer, self).__init__(aggr='add')  # "Add" aggregation 
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        print(x)
        return self.propagate(edge_index, x=x)

    def message(self, x_j):
        # x_j has shape [E, out_channels]

        return  x_j

gcn_layer = GraphConvLayer()
edge_index = torch.tensor([[0, 0, 1, 2,3,3],
                           [1, 3, 0, 0,2,1]], dtype=torch.long)
x = torch.tensor(X.astype(np.float32))
x.shape
gcn_layer(x, edge_index)

ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
W = np.array([[0.4, -2,0.6,1.2],
             [0.3, -2,0.7,-1.3]])
transformed_features = X @ W
print(transformed_features)
new_features  = A @ transformed_features
print(new_features)

In [ ]:
class GraphConvLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GraphConvLayer, self).__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.lin = torch.nn.Linear(in_channels, out_channels, bias = False)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        x = self.lin(x)
        print(x)
        return self.propagate(edge_index, x=x)

    def message(self, x_j):
        # x_j has shape [E, out_channels]
        #print(x_j)

        return  x_j

gcn_layer = GraphConvLayer(2,4)

gcn_layer.lin.weight.data =  torch.Tensor(W.T)
gcn_layer.lin.weight.data
edge_index = torch.tensor([[0, 0, 1, 2,3,3],
                           [1, 3, 0, 0,2,1]], dtype=torch.long)
x = torch.tensor(X.astype(np.float32))
x.shape
gcn_layer(x, edge_index)

In [7]:
A = np.array([
    [0, 1, 1, 0],
    [1, 0, 0, 1], 
    [0, 0, 0, 1],
    [1, 0, 0, 0]],
    dtype=float
)
I = np.matrix(np.eye(A.shape[0]))
print("old A")
print(A)
A = A + I
print("Adjacency with self loops")
print(A)
print(A * X  * W)

old A
[[0. 1. 1. 0.]
 [1. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]
Adjacency with self loops
[[1. 1. 1. 0.]
 [1. 1. 0. 1.]
 [0. 0. 1. 1.]
 [1. 0. 0. 1.]]
[[  6.6 -40.   13.4 -11. ]
 [  9.1 -56.   18.9 -18.9]
 [ 10.3 -64.   21.7 -24.1]
 [  7.1 -44.   14.9 -16.1]]


In [8]:
class GraphConvLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GraphConvLayer, self).__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.lin = torch.nn.Linear(in_channels, out_channels, bias = False)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        # Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        print(edge_index)

        x = self.lin(x)
        #print(x)
        return self.propagate(edge_index, x=x)

    def message(self, x_j):
        # x_j has shape [E, out_channels]

        return  x_j

gcn_layer = GraphConvLayer(2,4)
W = np.array([[0.4, -2,0.6,1.2],
             [0.3, -2,0.7,-1.3]])
gcn_layer.lin.weight.data =  torch.Tensor(W.T)
gcn_layer.lin.weight.data
edge_index = torch.tensor([[0, 0, 1, 2,3,3],
                           [1, 3, 0, 0,2,1]], dtype=torch.long)
x = torch.tensor(X.astype(np.float32))
x.shape
gcn_layer(x, edge_index)

tensor([[0, 0, 1, 2, 3, 3, 0, 1, 2, 3],
        [1, 3, 0, 0, 2, 1, 0, 1, 2, 3]])


tensor([[  6.6000, -40.0000,  13.4000, -11.0000],
        [  9.1000, -56.0000,  18.9000, -18.9000],
        [ 10.3000, -64.0000,  21.7000, -24.1000],
        [  7.1000, -44.0000,  14.9000, -16.1000]],
       grad_fn=<ScatterAddBackward>)

In [9]:
D = np.array(np.sum(A, axis=0))[0]
D = np.diag(D)
print(D)
print(np.linalg.inv(D) * A)
print(np.linalg.inv(D) * A * X * W)

[[3. 0. 0. 0.]
 [0. 2. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 3.]]
[[0.33333333 0.33333333 0.33333333 0.        ]
 [0.5        0.5        0.         0.5       ]
 [0.         0.         0.5        0.5       ]
 [0.33333333 0.         0.         0.33333333]]
[[  2.2        -13.33333333   4.46666667  -3.66666667]
 [  4.55       -28.           9.45        -9.45      ]
 [  5.15       -32.          10.85       -12.05      ]
 [  2.36666667 -14.66666667   4.96666667  -5.36666667]]


In [10]:
class GraphConvLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GraphConvLayer, self).__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.lin = torch.nn.Linear(in_channels, out_channels, bias = False)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        print(edge_index)

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)
        print(x)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype)
        print(deg)
        deg_inv = deg.pow(-1)
        norm = deg_inv[col]
        print(norm)
        #print(norm.view(-1,1))

        # Step 4-5: Start propagating messages.
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        #print(x_j)
        return  norm.view(-1,1)  * x_j

gcn_layer = GraphConvLayer(2,4)
gcn_layer.lin.weight.data =  torch.Tensor(W.T)
print(gcn_layer.lin.weight.data)
edge_index = torch.tensor([[0, 0, 1, 2,3,3],
                           [1, 3, 0, 0,2,1]], dtype=torch.long)
x = torch.tensor(X.astype(np.float32))
x.shape
gcn_layer(x, edge_index)

tensor([[ 0.4000,  0.3000],
        [-2.0000, -2.0000],
        [ 0.6000,  0.7000],
        [ 1.2000, -1.3000]])
tensor([[0, 0, 1, 2, 3, 3, 0, 1, 2, 3],
        [1, 3, 0, 0, 2, 1, 0, 1, 2, 3]])
tensor([[  0.7000,  -4.0000,   1.3000,  -0.1000],
        [  2.0000, -12.0000,   4.0000,  -2.8000],
        [  3.9000, -24.0000,   8.1000,  -8.1000],
        [  6.4000, -40.0000,  13.6000, -16.0000]], grad_fn=<MmBackward>)
tensor([3., 2., 2., 3.])
tensor([0.5000, 0.3333, 0.3333, 0.3333, 0.5000, 0.5000, 0.3333, 0.5000, 0.5000,
        0.3333])


tensor([[  2.2000, -13.3333,   4.4667,  -3.6667],
        [  4.5500, -28.0000,   9.4500,  -9.4500],
        [  5.1500, -32.0000,  10.8500, -12.0500],
        [  2.3667, -14.6667,   4.9667,  -5.3667]],
       grad_fn=<ScatterAddBackward>)